In [1]:
GPU_device_id = str(7)
model_id_save_as = 'learningcurve-caednn-easy-final'
architecture_id = '../hyperparameter_search/hyperparameter-search-results/CNN-kfoldseasy-final-1-reluupdate_33'
model_class_id = 'CNN1D'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_200keV_log10time_10000.npy'
difficulty_setting = 'easy'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000,]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice
from tensorflow.keras.models import load_model

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_cnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [5]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [4]:
model_features = load_features(architecture_id)
model_features.output_function = tf.nn.softmax

In [5]:
model_features.cnn_kernels = model_features.cnn_kernel
model_features.pool_sizes = model_features.pool_size
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.metrics = [f1]
model_features.dropout_rate = model_features.dropout_probability
model_features.output_function = tf.nn.softmax
model_features.input_dim = 1024

In [9]:
cae_model = load_model('./final-models-keras/caepretrain-easy-final.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


## Train network

# Scale input data

In [10]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [11]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [12]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        
        model = compile_model(
            build_cnn_model,
            model_features)
        model_weights = model.get_weights()
        model_weights_updated = model_weights[:]
        model_weights_updated[0:6] = cae_model.get_weights()[0:6]
        model.set_weights(model_weights_updated)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 50
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 2s 39ms/sample - loss: 4.8103 - f1: 0.0000e+00 - val_loss: 4.5658 - val_f1: 0.0000e+00
Epoch 2/2000
50/50 [==============================] - 0s 5ms/sample - loss: 4.4379 - f1: 0.0000e+00 - val_loss: 4.3522 - val_f1: 0.0000e+00
Epoch 3/2000
50/50 [==============================] - 0s 5ms/sample - loss: 4.1333 - f1: 0.0000e+00 - val_loss: 4.1507 - val_f1: 0.0064
Epoch 4/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8760 - f1: 0.0000e+00 - val_loss: 3.9441 - val_f1: 0.0256
Epoch 5/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.5871 - f1: 0.0588 - val_loss: 3.7293 - val_f1: 0.0731
Epoch 6/2000
50/50 [==============================] - 0s 5ms/sample - loss: 3.2734 - f1: 0.1857 - val_loss: 3.5255 - val_f1: 0.0978
Epoch 7/2000
50/50 [==

50/50 [==============================] - 0s 5ms/sample - loss: 0.4982 - f1: 1.0000 - val_loss: 1.7737 - val_f1: 0.5925
Epoch 58/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.4865 - f1: 1.0000 - val_loss: 1.7704 - val_f1: 0.5925
Epoch 59/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.4911 - f1: 1.0000 - val_loss: 1.7688 - val_f1: 0.5958
Epoch 60/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.4750 - f1: 1.0000 - val_loss: 1.7680 - val_f1: 0.6002
Epoch 61/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.4731 - f1: 1.0000 - val_loss: 1.7704 - val_f1: 0.5987
Epoch 62/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.4568 - f1: 1.0000 - val_loss: 1.7687 - val_f1: 0.5978
Running through fold 2
Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 1s 20ms/sample - loss: 4.8138 - f1: 0.0000e+00 - val_loss: 4.5884 - val_f1: 0.0000e+00
Epoch 

Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 1s 21ms/sample - loss: 4.7925 - f1: 0.0000e+00 - val_loss: 4.5595 - val_f1: 0.0000e+00
Epoch 2/2000
50/50 [==============================] - 0s 5ms/sample - loss: 4.4037 - f1: 0.0000e+00 - val_loss: 4.3325 - val_f1: 0.0000e+00
Epoch 3/2000
50/50 [==============================] - 0s 5ms/sample - loss: 4.0831 - f1: 0.0000e+00 - val_loss: 4.1081 - val_f1: 6.4475e-04
Epoch 4/2000
50/50 [==============================] - 0s 5ms/sample - loss: 3.7713 - f1: 0.0829 - val_loss: 3.8949 - val_f1: 0.0279
Epoch 5/2000
50/50 [==============================] - 0s 5ms/sample - loss: 3.4735 - f1: 0.1818 - val_loss: 3.7064 - val_f1: 0.0544
Epoch 6/2000
50/50 [==============================] - 0s 5ms/sample - loss: 3.2392 - f1: 0.1732 - val_loss: 3.5368 - val_f1: 0.0778
Epoch 7/2000
50/50 [==============================] - 0s 5ms/sample - loss: 2.9753 - f1: 0.2477 - val_loss: 3.3783 - val_f1: 0.1159
Epoch

...

Epoch 16/2000
5000/5000 [==============================] - 1s 299us/sample - loss: 0.2555 - f1: 0.9576 - val_loss: 0.2894 - val_f1: 0.9557
Epoch 17/2000
5000/5000 [==============================] - 1s 299us/sample - loss: 0.2577 - f1: 0.9599 - val_loss: 0.2681 - val_f1: 0.9583
Epoch 18/2000
5000/5000 [==============================] - 1s 298us/sample - loss: 0.2346 - f1: 0.9653 - val_loss: 0.2473 - val_f1: 0.9568
Epoch 19/2000
5000/5000 [==============================] - 1s 298us/sample - loss: 0.2248 - f1: 0.9680 - val_loss: 0.2143 - val_f1: 0.9594
Epoch 20/2000
5000/5000 [==============================] - 1s 300us/sample - loss: 0.2577 - f1: 0.9611 - val_loss: 0.2235 - val_f1: 0.9707
Epoch 21/2000
5000/5000 [==============================] - 1s 299us/sample - loss: 0.2099 - f1: 0.9701 - val_loss: 0.2284 - val_f1: 0.9655
Epoch 22/2000
5000/5000 [==============================] - 1s 297us/sample - loss: 0.2545 - f1: 0.9684 - val_loss: 0.2567 - val_f1: 0.9725
Epoch 23/2000
5000/5000 [==

Train on 5000 samples, validate on 3000 samples
Epoch 1/2000
5000/5000 [==============================] - 3s 692us/sample - loss: 1.7001 - f1: 0.6112 - val_loss: 0.8331 - val_f1: 0.8728
Epoch 2/2000
5000/5000 [==============================] - 2s 310us/sample - loss: 0.7033 - f1: 0.8912 - val_loss: 0.5999 - val_f1: 0.9100
Epoch 3/2000
5000/5000 [==============================] - 2s 310us/sample - loss: 0.5427 - f1: 0.9099 - val_loss: 0.4692 - val_f1: 0.9223
Epoch 4/2000
5000/5000 [==============================] - 2s 310us/sample - loss: 0.4472 - f1: 0.9286 - val_loss: 0.4425 - val_f1: 0.9248
Epoch 5/2000
5000/5000 [==============================] - 2s 311us/sample - loss: 0.4007 - f1: 0.9373 - val_loss: 0.4545 - val_f1: 0.9217
Epoch 6/2000
5000/5000 [==============================] - 2s 310us/sample - loss: 0.3582 - f1: 0.9445 - val_loss: 0.3347 - val_f1: 0.9421
Epoch 7/2000
5000/5000 [==============================] - 2s 310us/sample - loss: 0.3186 - f1: 0.9523 - val_loss: 0.3163 - v

10000/10000 [==============================] - 5s 497us/sample - loss: 1.2084 - f1: 0.7521 - val_loss: 0.5688 - val_f1: 0.9159
Epoch 2/2000
10000/10000 [==============================] - 3s 287us/sample - loss: 0.4957 - f1: 0.9192 - val_loss: 0.4042 - val_f1: 0.9370
Epoch 3/2000
10000/10000 [==============================] - 3s 290us/sample - loss: 0.3943 - f1: 0.9351 - val_loss: 0.4004 - val_f1: 0.9441
Epoch 4/2000
10000/10000 [==============================] - 3s 286us/sample - loss: 0.3370 - f1: 0.9446 - val_loss: 0.3113 - val_f1: 0.9496
Epoch 5/2000
10000/10000 [==============================] - 3s 284us/sample - loss: 0.3047 - f1: 0.9486 - val_loss: 0.3327 - val_f1: 0.9435
Epoch 6/2000
10000/10000 [==============================] - 3s 284us/sample - loss: 0.2858 - f1: 0.9529 - val_loss: 0.3006 - val_f1: 0.9523
Epoch 7/2000
10000/10000 [==============================] - 3s 286us/sample - loss: 0.2674 - f1: 0.9578 - val_loss: 0.2191 - val_f1: 0.9700
Epoch 8/2000
10000/10000 [=======

15000/15000 [==============================] - 4s 284us/sample - loss: 0.1240 - f1: 0.9951 - val_loss: 0.0646 - val_f1: 0.9985
Epoch 19/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.1986 - f1: 0.9874 - val_loss: 0.2581 - val_f1: 0.9865
Epoch 20/2000
15000/15000 [==============================] - 4s 285us/sample - loss: 0.0876 - f1: 0.9993 - val_loss: 0.0589 - val_f1: 1.0000
Running through fold 2
Train on 15000 samples, validate on 3000 samples
Epoch 1/2000
15000/15000 [==============================] - 7s 457us/sample - loss: 0.9713 - f1: 0.8098 - val_loss: 0.5557 - val_f1: 0.9038
Epoch 2/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.4131 - f1: 0.9321 - val_loss: 0.3418 - val_f1: 0.9456
Epoch 3/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3316 - f1: 0.9446 - val_loss: 0.2713 - val_f1: 0.9529
Epoch 4/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3002 - f1: 0.

Epoch 13/2000
15000/15000 [==============================] - 4s 291us/sample - loss: 0.1520 - f1: 0.9916 - val_loss: 0.3512 - val_f1: 0.9787
Epoch 14/2000
15000/15000 [==============================] - 4s 292us/sample - loss: 0.1565 - f1: 0.9927 - val_loss: 0.1560 - val_f1: 0.9902
Epoch 15/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.1132 - f1: 0.9950 - val_loss: 0.0899 - val_f1: 0.9972
Epoch 16/2000
15000/15000 [==============================] - 4s 287us/sample - loss: 0.1559 - f1: 0.9928 - val_loss: 0.1022 - val_f1: 0.9960
Epoch 17/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.1170 - f1: 0.9952 - val_loss: 0.1852 - val_f1: 0.9963
Epoch 18/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.1327 - f1: 0.9954 - val_loss: 0.0667 - val_f1: 0.9988
Epoch 19/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.1577 - f1: 0.9925 - val_loss: 0.1163 - val_f1: 0.9985


Running thr